https://www.yu.edu/sitemap.xml

## Import Liraries

In [2]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
import pinecone
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
from langchain import HuggingFacePipeline
from huggingface_hub import notebook_login
import torch

## Extract data from URL

In [3]:
URLs = [
    # "https://ineuron.ai/"
    "https://www.yu.edu/"
]

In [4]:
loader = UnstructuredURLLoader(urls=URLs)
data = loader.load()

KeyboardInterrupt: 

: 

In [ ]:
data

In [ ]:
loader = UnstructuredURLLoader(urls = URLs)
data = loader.load()

text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=1000,
                                    chunk_overlap=200)

text_chunks=text_splitter.split_documents(data)
len(text_chunks)

embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

query_result = embeddings.embed_query("Hello world")
len(query_result)


import os

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', 'a31b63c6-9ceb-427d-a12c-9c080a92a818')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')



import pinecone
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "website-bot" # put in the name of your pinecone index here

docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)



tokenizer = AutoTokenizer.from_pretrained("daryl149/llama-2-7b-chat-hf",
                                          # "meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)


model = AutoModelForCausalLM.from_pretrained("daryl149/llama-2-7b-chat-hf",
                                            # "meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                              load_in_8bit=True,
                                              #load_in_4bit=True
                                             )



pipe = transformers.pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )


llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

query = "Tell me about the yeshiva university AI program."
qa.invoke(query)